In [ ]:
pwd

In [1]:
%matplotlib inline
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import os
import sys
import cv2
import matplotlib.pyplot as plt
import glob
sys.path.append('../../Vision/DataManagement')

import utils.inference.inference as infer
import utils.inference.inference as infer
from utils.inference.inference import MMdetDocker, RunInference
os.chdir('/home/ubuntu/mmdetection')

In [10]:
EXP = 'exp20'
CFG_NAME = 'faster_rcnn_r50_caffe_fpn_mstrain_1x_coco-car_detector.py'
EPOCH = 'epoch_20.pth'
PORT = 8090

In [11]:
root_model = f"/home/ubuntu/mmdetection/work_dirs/{EXP}" 
cfg_path = f"{root_model}/{CFG_NAME}"
model_path = f"{root_model}/{EPOCH}"
output_folder = f"/home/ubuntu/Vision/DataManagement/utils/inference/{EXP}"
model_name = EXP

In [12]:
inference_path, model_name = infer.convert_model(cfg_path, model_path, output_folder, model_name)

/opt/conda/envs/open-mmlab/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
ERROR:root:/home/ubuntu/Vision/DataManagement/utils/inference/exp20/exp20.mar already exists.
Please specify --force/-f option to overwrite the model archive output file.
See -h/--help for more details.


In [13]:
docker = MMdetDocker(root=f"/home/ubuntu/Vision/DataManagement/utils/inference", inference_path=model_name, model_name=model_name, base_port = PORT, print_docker_exec = True)

In [14]:
docker.build_mmdet_docker()

sha256:4f76867ff3c29257442286cdd1c6dcb65d6587b0248678665e5a4230ae831109


In [15]:
docker.run_mmdet_docker()

docker run --rm --cpus 8 -p8090:8080 -p8091:8081 -p8092:8082 --mount type=bind,source=/home/ubuntu/Vision/DataManagement/utils/inference/exp20,target=/home/model-server/model-store mmdet-serve:latest


In [9]:
test_images = glob.glob(f"/home/ubuntu/mmdetection/work_dirs/exp20/results/*.jpg")
len(test_images), test_images[:2]

(199,
 ['/home/ubuntu/mmdetection/work_dirs/exp20/results/972_98_00780796_1.jpg',
  '/home/ubuntu/mmdetection/work_dirs/exp20/results/972_00226317_1.jpg'])

In [ ]:
images_names = cu.clean_names_list(test_images)

In [ ]:
def plot_predictions(result, image):
    x1, y1, x2, y2 = map(int, result['bbox'])
    score = "{:.2f}".format(result['score'])
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)
    image = cv2.putText(image,
                        score, 
                        (x1, y1),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.2,
                        (0, 0, 0),
                        1,
                        cv2.LINE_AA, False)
    return image

In [ ]:
for test_image in test_images:
    infer1 = RunInference(url_serve='http://127.0.0.1:8080/predictions', image_path=test_image, model_name='exp13')
    infer2 = RunInference(url_serve='http://127.0.0.1:8090/predictions', image_path=test_image, model_name='exp20')
    predictions1 = infer1.get_predicions()
    predictions2 = infer2.get_predicions()
    results1 = infer1.get_predicions()
    results2 = infer2.get_predicions()
    image = cv2.imread(f"/home/ubuntu/Vision/DataManagement/utils/inference/0_111_02207063_1.jpg")
    for result in results:
        image = plot_predictions(result, image)
    plt.figure(figsize=(10,10)); plt.imshow(image); plt.show()

In [ ]:
infer = RunInference(url_serve='http://127.0.0.1:8090/predictions', image_path=f"/home/ubuntu/Vision/images/raw/98_00780796.jpg", model_name='exp20')

In [ ]:
predictions = infer.get_predicions()

In [ ]:
results = infer.get_predicions()
len(results), results[:2]

In [ ]:
image = cv2.imread(f"/home/ubuntu/Vision/images/raw/98_00780796.jpg")
for result in results:
    x1, y1, x2, y2 = map(int, result['bbox'])
    score = "{:.2f}".format(result['score'])
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)
    image = cv2.putText(image,
                        score, 
                        (x1, y1),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.2,
                        (0, 0, 0),
                        1,
                        cv2.LINE_AA, False)
plt.figure(figsize=(40,40)); plt.imshow(image); plt.show()

In [ ]:
import utils.inference.detect as det

In [ ]:
os.chdir(f"/home/ubuntu/Vision/DataManagement/utils/inference")

In [ ]:
det.inference_yolov5_on_cloud('0_111_02207063_1.jpg')

In [ ]:
pwd

In [ ]:
import requests

def yolo_im_encoder(image_path):
    im = cv2.imread(image_path)[:,:,::-1]
    _, x =cv2.imencode('.png', im,)
    x_bites = x.tobytes()
    return x_bites


def yolo_get_predicions(image_path):
    url_serve = "http://odworkersgpu-asg-1-1734987265.eu-central-1.elb.amazonaws.com/detect-img"
    header = {"content-type": "image"}
    # data  = yolo_im_encoder(image_path)
    response = requests.post(url=url_serve, files={"file": ("filename", open(image_path, "rb"), "image/jpeg")}, json={})
    return response.json()[0]

In [ ]:
pred = yolo_get_predicions('0_111_02207063_1.jpg')
pred

In [ ]:
image = cv2.imread(f"0_111_02207063_1.jpg")
for result in pred:
    x1, y1, x2, y2 = map(int, result[:-1])
    score = "{:.2f}".format(result[-1])
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)
    image = cv2.putText(image,
                        score, 
                        (x1, y1),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.2,
                        (0, 0, 0),
                        1,
                        cv2.LINE_AA, False)
plt.figure(figsize=(10,10)); plt.imshow(image); plt.show()

In [ ]:
from mmcv import Config, DictAction
from mmdet.utils import (collect_env, get_device, get_root_logger,
                         replace_cfg_vals, setup_multi_processes,
                         update_data_root)

In [ ]:
pwd

In [ ]:
cfg_path = '../work_dirs/exp20/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco-car_detector.py'
chckpnt_path = '../work_dirs/exp20/epoch_20.pth'

In [ ]:
cfg = Config.fromfile('../work_dirs/exp20/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco-car_detector.py')
cfg = replace_cfg_vals(cfg)
update_data_root(cfg)

In [ ]:
cfg.keys()

In [ ]:
!python ../tools/deployment/pytorch2onnx.py {cfg_path} {chckpnt_path} --output-file ./epoch_20.onnx --verify --input-img ../tests/data/color.jpg